In [1]:
%env THEANO_FLAGS="device=gpu4"

env: THEANO_FLAGS="device=gpu4"


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm_notebook as tqdm
from PIL import Image

from librosa import load, logamplitude
from librosa.feature import melspectrogram

In [5]:
from CoolSoundNetwork import Network

In [3]:
df = pd.DataFrame.from_csv("pronuns.csv")

In [45]:
accent_count = len(df.accent.unique())

## Deep Learning

In [60]:
import theano
import theano.tensor as T

import lasagne

In [61]:
input_tensor = T.matrix("Vector input")
target_gender = T.ivector("Target gender")
target_accent = T.ivector("Target acent")

In [62]:
input_ = lasagne.layers.InputLayer((None, 100), input_var=input_tensor, name="Network input")
batch_norm0 = lasagne.layers.batch_norm(input_, name="Batch normalization")

### Gender

In [63]:
dense0 = lasagne.layers.DenseLayer(batch_norm0, 50, name="Dense 0")
dense1 = lasagne.layers.DenseLayer(dense0, 20, name="Dense 1")
gender_out = lasagne.layers.DenseLayer(dense1, 1, nonlinearity=lasagne.nonlinearities.sigmoid, name="Output")

In [64]:
gender_out.output_shape

(None, 1)

In [65]:
gender_predicted = lasagne.layers.get_output(gender_out)
gender_param = lasagne.layers.get_all_params(gender_out, trainable=True)

In [66]:
gender_loss = lasagne.objectives.binary_crossentropy(gender_predicted, target_gender).mean()

gender_updates = lasagne.updates.adagrad(gender_loss, gender_param)

### Acent

In [67]:
dense0 = lasagne.layers.DenseLayer(batch_norm0, 200, name="Dense 0")
accent_out = lasagne.layers.DenseLayer(dense0, accent_count,
                                       nonlinearity=lasagne.nonlinearities.softmax, name="Accent output")

In [68]:
accent_out.output_shape

(None, 111)

In [69]:
accent_predicted = lasagne.layers.get_output(accent_out)
accent_param = lasagne.layers.get_all_params(accent_out, trainable=True)

In [70]:
accent_loss = lasagne.objectives.binary_crossentropy(accent_predicted, target_accent).mean()

accent_updates = lasagne.updates.adagrad(accent_loss, accent_param)

### Compile

In [71]:
gender_train = theano.function([input_.input_var, target_gender], updates=gender_updates)
accent_train = theano.function([input_.input_var, target_accent], updates=accent_updates)

gender_predict = theano.function([input_.input_var], gender_predicted)
accent_predict = theano.function([input_.input_var], accent_predicted)

predict = theano.function([input_.input_var], [gender_predicted, accent_predicted])